In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from tqdm import tqdm
import pandas as pd
import time

import datetime

In [13]:
def num_of_reviews(location, food_category):
    location
    name = location.split(" ")[1]
    list_url = "https://www.yogiyo.co.kr/mobile/#/"
    urls = []
    df = pd.DataFrame(columns=['url', '개수충족'])

    #### 주소 기준으로 초기화
    driver = webdriver.Chrome('/Users/jijoonghong/Downloads/chromedriver')

    driver.get(list_url)
    time.sleep(8)
    element = driver.find_element_by_name("address_input")
    element.clear()
    element.send_keys(location)
    btn = driver.find_element_by_css_selector("#button_search_address > button.btn.btn-default.ico-pick")
    btn.click()
    time.sleep(2)
    
    # 리뷰 많은 순으로 sorting
    driver.find_element_by_xpath('//*[@id="content"]/div/div[1]/div[2]/div/select').click()
    driver.find_element_by_xpath('//*[@id="content"]/div/div[1]/div[2]/div/select/option[3]').click()
    driver.find_element_by_xpath('//*[@id="content"]/div/div[1]/div[2]/div/select').click()
    time.sleep(3)
    
    # 카테고리 접속 및 가게 리스트 
    driver.get(list_url + food_category)
    
    last_height = driver.execute_script("return document.body.scrollHeight")
    print(last_height)

    while True:

        scroll_down = 0
        while scroll_down < 10:
            element.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.2)
            scroll_down += 1

        new_height = driver.execute_script("return document.body.scrollHeight")
        time.sleep(1)
        if new_height == last_height:
            print("끝")
            break

        last_height = new_height
    
    store_list = driver.find_elements_by_xpath("//div[@class='item clearfix']")
    
    return len(store_list)

In [2]:
def get_url3(location, food_category, start, end):
    name = location.split(" ")[1]
    list_url = "https://www.yogiyo.co.kr/mobile/#/"
    urls = []
    parsed_data = pd.read_csv("./강남구url/강남구_"+food_category+"_url.csv")
    stores = parsed_data['가게명'].tolist()
    df = pd.DataFrame(columns=['url', '개수충족','가게명'])
    
    #### 주소 기준으로 초기화
    driver = webdriver.Chrome('/Users/jijoonghong/Downloads/chromedriver')

    driver.get(list_url)
    time.sleep(8)
    element = driver.find_element_by_name("address_input")
    element.clear()
    element.send_keys(location)
    btn = driver.find_element_by_css_selector("#button_search_address > button.btn.btn-default.ico-pick")
    btn.click()
    time.sleep(2)
    
    # 리뷰 많은 순으로 sorting
    driver.find_element_by_xpath('//*[@id="content"]/div/div[1]/div[2]/div/select').click()
    driver.find_element_by_xpath('//*[@id="content"]/div/div[1]/div[2]/div/select/option[3]').click()
    driver.find_element_by_xpath('//*[@id="content"]/div/div[1]/div[2]/div/select').click()
    time.sleep(3)
    
    # 카테고리 접속 및 가게 리스트 
    driver.get(list_url + food_category) 
    
    last_height = driver.execute_script("return document.body.scrollHeight")


    try:
        for i in tqdm(range(start, end)):
            time.sleep(0.5)
            is_valid = 1
            path = '//*[@id="content"]/div/div[5]/div/div/div[' + str(i) + ']/div'
            
            try:
                store_name = driver.find_element_by_xpath('//*[@id="content"]/div/div[5]/div/div/div['+str(i)+']/div/table/tbody/tr/td[2]/div/div[1]').text

            except:
                for j in range(int(i // 60)):
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    if i<480:
                        time.sleep(i//60)
                    else:
                        time.sleep(8)
            try:
                store_name = driver.find_element_by_xpath('//*[@id="content"]/div/div[5]/div/div/div['+str(i)+']/div/table/tbody/tr/td[2]/div/div[1]').text
                n = driver.find_element_by_css_selector(
                    "#content > div > div:nth-child(5) > div > div > div:nth-child(" + str(
                        i) + ") > div > table > tbody > tr > td:nth-child(2) > div > div.stars > span:nth-child(2)").text

            except:
                time.sleep(6)
                store_name = driver.find_element_by_xpath('//*[@id="content"]/div/div[5]/div/div/div['+str(i)+']/div/table/tbody/tr/td[2]/div/div[1]').text
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                n = driver.find_element_by_css_selector(
                    "#content > div > div:nth-child(5) > div > div > div:nth-child(" + str(
                        i) + ") > div > table > tbody > tr > td:nth-child(2) > div > div.stars > span:nth-child(2)").text
            print(store_name)
            if store_name in stores:
                print("skip")
                continue
                
            some_tag = driver.find_element_by_xpath(path)
            if n == "" or int(n.split(" ")[1]) < 10:
                is_valid = 0

            # somthing element 까지 스크롤
            action = ActionChains(driver)
            action.move_to_element(some_tag).perform()
            driver.find_element_by_xpath(path).click()
            time.sleep(2)

            row = pd.DataFrame([(driver.current_url, is_valid, store_name)],columns=['url', '개수충족','가게명'])
            print(row)
            df = df.append(row)

            driver.back()
            time.sleep(3)
    except:
        df.to_csv("./temp/"+location.split(" ")[1]+"_"+food_category+"_url_list2_비정상종료-"+str(start)+".csv",
              encoding='utf-8-sig')
        
    df.to_csv("./temp/"+location.split(" ")[1]+"_"+food_category+"_url_list2-"+str(start)+".csv",
              encoding='utf-8-sig')
    driver.close()

In [ ]:
def get_name(category):
    driver = webdriver.Chrome('/Users/jijoonghong/Downloads/chromedriver')
    df = pd.read_csv("./강남구_"+category+"_url_list.csv")
    urls = df['url'].tolist()
    store_name = []
    for url in urls:
        driver.get(url)
        time.sleep(1.5)
        try:
            store_name.append(driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/div[1]/div[1]/span').text)
        except:
            print(url)
    df['가게명']=store_name
    df.to_csv("./강남구_"+category+"_url_list.csv", encoding='utf-8-sig')

In [ ]:
dic = {}
categories = ["프랜차이즈", "한식", "치킨", "피자양식", "중식", "일식돈까스", "족발보쌈", "야식", "분식", "카페디저트", "편의점"]

for cat in categories:
    dic[cat] = num_of_reviews("서울특별시 강남구 삼성동 16-1 강남구청", cat)
    
'''
결과 

dic = {'1인분주문': 530,
 '프랜차이즈': 818,
 '치킨': 331,
 '피자양식': 571,
 '중식': 210,
 '한식': 1614,
 '일식돈까스': 607,
 '족발보쌈': 126,
 '야식': 677,
 '분식': 666,
 '카페디저트': 727,
 '편의점': 80}

'''

# 카테고리별로 url 수집
# 맨 아래까지 스크롤 시 60개씩 동적으로 가게 추가 표시되어 컴퓨터에 상당한 무리가 가며, 해당 가게에 접속하여 url 수집 후 되돌아오면 세션이 초기화 됨
# 에러 핸들링을 위하여 100개씩 나누어서 수집
for cat in categories:
    review_num = dic[cat]
    temp = review_num
    n = 1
    for i in range(review_num//100+1):
        print(n, temp)
        if temp < 100:
            get_url3("서울특별시 강남구 삼성동 16-1 강남구청", cat, n, review_num+1)
            break
        get_url3("서울특별시 강남구 삼성동 16-1 강남구청", cat, n, n+100)
        n+=100
        temp = review_num - n
        

# 수집된 url 데이터셋에 가게이름 추가
for category in categories:
    try:
        get_name(category)
    except:
        print(category)
        continue